In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *


import os
import sys

project_pth=os.path.join(os.getcwd(),"..","..")
sys.path.append(project_pth)

from utils.transformations import *

### AutoLoader

In [0]:
df_user = spark.readStream.format("cloudFiles").option("cloudFiles.format", "parquet").option("cloudFiles.schemaLocation","abfss://silver@spotifystoragerohan.dfs.core.windows.net/DimUser/checkpoint").option("clodFiles.schemaEvolutionMode", "addNewColumns").load("abfss://bronze@spotifystoragerohan.dfs.core.windows.net/DimUser")

In [0]:
df_user.display()

In [0]:
df_user=df_user.withColumn("user_name",upper(col("user_name")))

In [0]:
df_user_obj=reusable()
df_user=df_user_obj.dropColumns(df_user,['_rescued_data'])
display(df_user)

In [0]:
df_user.writeStream.format("delta").outputMode("append").option("checkpointLocation","abfss://silver@spotifystoragerohan.dfs.core.windows.net/DimUser/checkpoint").trigger(once=True).start("abfss://silver@spotifystoragerohan.dfs.core.windows.net/DimUser/data")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS spotify_cata.silver.dimuser
USING DELTA
LOCATION 'abfss://silver@spotifystoragerohan.dfs.core.windows.net/DimUser/data';

### DimArtist

In [0]:
df_artist = spark.readStream.format("cloudFiles").option("cloudFiles.format", "parquet").option("cloudFiles.schemaLocation","abfss://silver@spotifystoragerohan.dfs.core.windows.net/DimArtist/checkpoint").option("clodFiles.schemaEvolutionMode", "addNewColumns").load("abfss://bronze@spotifystoragerohan.dfs.core.windows.net/DimArtist")

In [0]:
df_artist=reusable().dropColumns(df_artist,['_rescued_data'])
df_artist=df_artist.dropDuplicates(['artist_id'])
display(df_artist)

In [0]:
df_artist.writeStream.format("delta").outputMode("append").option("checkpointLocation","abfss://silver@spotifystoragerohan.dfs.core.windows.net/DimArtist/checkpoint").trigger(once=True).start("abfss://silver@spotifystoragerohan.dfs.core.windows.net/DimArtist/data")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS spotify_cata.silver.dimartist
USING DELTA
LOCATION 'abfss://silver@spotifystoragerohan.dfs.core.windows.net/DimArtist/data';

### DimTrack

In [0]:
df_track = spark.readStream.format("cloudFiles").option("cloudFiles.format", "parquet").option("cloudFiles.schemaLocation","abfss://silver@spotifystoragerohan.dfs.core.windows.net/DimTrack/checkpoint").option("clodFiles.schemaEvolutionMode", "addNewColumns").load("abfss://bronze@spotifystoragerohan.dfs.core.windows.net/DimTrack")

In [0]:
df_track=df_track.withColumn("durationFlag",when(col("duration_sec")>150,"low").when(col("duration_sec")<300,"medium").otherwise("high"))
df_track=df_track.withColumn("track_name",regexp_replace(col('track_name'),'-',' '))
df_track=reusable().dropColumns(df_track,['_rescued_data'])


In [0]:
df_track.writeStream.format("delta").outputMode("append").option("checkpointLocation","abfss://silver@spotifystoragerohan.dfs.core.windows.net/DimTrack/checkpoint").trigger(once=True).start("abfss://silver@spotifystoragerohan.dfs.core.windows.net/DimTrack/data")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS spotify_cata.silver.dimtrack
USING DELTA
LOCATION 'abfss://silver@spotifystoragerohan.dfs.core.windows.net/DimTrack/data';

### DimDate


In [0]:
df_date = spark.readStream.format("cloudFiles").option("cloudFiles.format", "parquet").option("cloudFiles.schemaLocation","abfss://silver@spotifystoragerohan.dfs.core.windows.net/DimDate/checkpoint").option("clodFiles.schemaEvolutionMode", "addNewColumns").load("abfss://bronze@spotifystoragerohan.dfs.core.windows.net/DimDate")

In [0]:
df_date=reusable().dropColumns(df_date,['_rescued_data'])

In [0]:
df_date.writeStream.format("delta").outputMode("append").option("checkpointLocation","abfss://silver@spotifystoragerohan.dfs.core.windows.net/DimDate/checkpoint").trigger(once=True).start("abfss://silver@spotifystoragerohan.dfs.core.windows.net/DimDate/data")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS spotify_cata.silver.dimdate
USING DELTA
LOCATION 'abfss://silver@spotifystoragerohan.dfs.core.windows.net/DimDate/data';

### FactStream

In [0]:
df_fact = spark.readStream.format("cloudFiles").option("cloudFiles.format", "parquet").option("cloudFiles.schemaLocation","abfss://silver@spotifystoragerohan.dfs.core.windows.net/FactStream/checkpoint").option("clodFiles.schemaEvolutionMode", "addNewColumns").load("abfss://bronze@spotifystoragerohan.dfs.core.windows.net/FactStream")

In [0]:
df_fact=reusable().dropColumns(df_fact,['_rescued_data'])

In [0]:
df_fact.writeStream.format("delta").outputMode("append").option("checkpointLocation","abfss://silver@spotifystoragerohan.dfs.core.windows.net/FactStream/checkpoint").trigger(once=True).start("abfss://silver@spotifystoragerohan.dfs.core.windows.net/FactStream/data")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS spotify_cata.silver.factstream
USING DELTA
LOCATION 'abfss://silver@spotifystoragerohan.dfs.core.windows.net/FactStream/data';

In [0]:
%sql
SELECT * FROM spotify_cata.gold.dimtrack
WHERE track_id IN (46,5)